In [40]:
import pandas as pd

# Load the dataset
file_path = 'data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,subject#,age,sex,test_time,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,Shimmer,...,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE,motor_UPDRS,total_UPDRS
0,1,72,0,5.6431,0.00662,0.000034,0.00401,0.00317,0.01204,0.02565,...,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006,28.199,34.398
1,1,72,0,12.6660,0.00300,0.000017,0.00132,0.00150,0.00395,0.02024,...,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810,28.447,34.894
2,1,72,0,19.6810,0.00481,0.000025,0.00205,0.00208,0.00616,0.01675,...,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014,28.695,35.389
3,1,72,0,25.6470,0.00528,0.000027,0.00191,0.00264,0.00573,0.02309,...,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277,28.905,35.810
4,1,72,0,33.6420,0.00335,0.000020,0.00093,0.00130,0.00278,0.01703,...,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361,29.187,36.375


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features and Targets
X = data.drop(['subject#','motor_UPDRS', 'total_UPDRS'], axis=1)
y = data[['motor_UPDRS', 'total_UPDRS']]

# Normalizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the dataset into training and testing sets (70-30 split)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Checking the shape of the datasets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((4112, 19), (1763, 19), (4112, 2), (1763, 2))

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Building the Neural Network model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # First hidden layer
model.add(Dense(64, activation='relu'))  # Second hidden layer
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))  # Third hidden layer
model.add(Dense(y_train.shape[1], activation='linear'))  # Output layer

# Compiling the model
model.compile(optimizer=Adam(), loss='mean_squared_error')


In [46]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=200, batch_size=32)


Epoch 1/200


103/103 [==============================] - 0s 2ms/step - loss: 8.9805 - val_loss: 21.5242
Epoch 2/200
103/103 [==============================] - 0s 1ms/step - loss: 9.1209 - val_loss: 21.6039
Epoch 3/200
103/103 [==============================] - 0s 2ms/step - loss: 9.8640 - val_loss: 23.4547
Epoch 4/200
103/103 [==============================] - 0s 1ms/step - loss: 9.2554 - val_loss: 24.3554
Epoch 5/200
103/103 [==============================] - 0s 1ms/step - loss: 9.4739 - val_loss: 22.4772
Epoch 6/200
103/103 [==============================] - 0s 1ms/step - loss: 8.7729 - val_loss: 21.5586
Epoch 7/200
103/103 [==============================] - 0s 1ms/step - loss: 8.5729 - val_loss: 22.4253
Epoch 8/200
103/103 [==============================] - 0s 1ms/step - loss: 9.5238 - val_loss: 20.3736
Epoch 9/200
103/103 [==============================] - 0s 1ms/step - loss: 8.1548 - val_loss: 21.2165
Epoch 10/200
103/103 [==============================] - 0s 1ms/step - loss: 8.7826 - val_loss:

In [47]:
test_loss = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)


 1/56 [..............................] - ETA: 0s - loss: 30.6331

56/56 [==============================] - 0s 795us/step - loss: 14.3893
Test Loss: 14.389286041259766


In [48]:
from sklearn.metrics import r2_score

# Predicting using the test data
y_pred = model.predict(X_test)

# Calculating R-squared value
r_squared = r2_score(y_test, y_pred)
print("R-squared Value:", r_squared)


 1/56 [..............................] - ETA: 1s

56/56 [==============================] - 0s 1ms/step
R-squared Value: 0.8373509752669708
